# Kite
SIMcards with and without APN

In [1]:
import pandas as pd
import numpy as np
# Import datetime from the datetime module
from datetime import datetime, timedelta
import matplotlib.pyplot as plt   
import seaborn as sns

In [2]:
# Adjust pandas display and formatting settings

# Remove scientific notations and display numbers with 2 decimal points instead
pd.options.display.float_format = '{:,.2f}'.format        

# Increase cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

# Update default style and size of charts
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [10, 5]

## Data

### Transform data from Kite

In [3]:
def getDataFromKite(csvFile):
    df = pd.read_csv(csvFile, sep=';', low_memory=False)
    df = df[['icc', 'imsi', 'msisdn', 'lifeCycleStatus','currentApn', 'currentIp',
             'gprsStatus_lastConnStart', 'gprsStatus_lastConnStop',
            'apn_apn1',
            'apn_apn2',
            'apn_apn3',
            'apn_apn4',
            'apn_apn5',
            'apn_apn6',
            'apn_apn7',
            'apn_apn8',
            'apn_apn9',
            'apn_apn10'
        ]]
    df = df.applymap(str) # change all columns to string type
    df = df.applymap(lambda x: x.lstrip('=""').rstrip('"')) # remove unnecessary characteres
    
    return df

In [4]:
# dftest = getDataFromKite('data/example.csv')
# list(dftest.columns)

In [5]:
%%time
df1 = getDataFromKite('data/SimInventoryExport_2021-06-08T12.41.22Z_1bxwtmd-23jsalj1j9a-60bf65720c06.0.csv')
df1.shape

CPU times: user 46.1 s, sys: 2.87 s, total: 49 s
Wall time: 53.8 s


(1000000, 18)

In [6]:
%%time
df2 = getDataFromKite('data/SimInventoryExport_2021-06-08T12.41.22Z_1bxwtmd-23jsalj1j9a-60bf65720c06.1.csv')
df2.shape

CPU times: user 38.2 s, sys: 2.36 s, total: 40.6 s
Wall time: 44.3 s


(899222, 18)

### Concat dataframes

In [7]:
df = pd.concat([df1, df2])
df.shape

(1899222, 18)

In [8]:
df.shape[0] == df1.shape[0] + df2.shape[0]

True

### Unique values

In [9]:
list(df['lifeCycleStatus'].unique())

['ACTIVATED', 'SUSPENDED', 'DEACTIVATED', 'INACTIVE_NEW', 'ACTIVATION_PENDANT']

In [10]:
pd.DataFrame(df['lifeCycleStatus'].value_counts())

,lifeCycleStatus
ACTIVATED,1261194
SUSPENDED,601069
INACTIVE_NEW,36889
ACTIVATION_PENDANT,64
DEACTIVATED,6


In [11]:
list(df['currentApn'].unique())

['nan',
 'pagseguro.m2m.vivo.com.br',
 'apnsmart.pagseguro.uol',
 'smart.m2m.vivo.com.br',
 'apn.pagseguro.uol.m2m.com.br']

In [12]:
pd.DataFrame(df['currentApn'].value_counts())

,currentApn
nan,1728875
pagseguro.m2m.vivo.com.br,157005
apnsmart.pagseguro.uol,11192
smart.m2m.vivo.com.br,1309
apn.pagseguro.uol.m2m.com.br,841


### Time

In [13]:
def convertTime(date_str):
    if date_str == 'nan':
        return datetime.now()
    date_format = "%Y-%m-%dT%H:%M:%S.%fZ"
    dt = datetime.strptime(date_str, date_format)
    return dt

In [14]:
# str1 = '2020-03-13T14:31:33.000Z'
# str2 = '2020-03-13T16:00:09.000Z'
# dt1 = convertTime(str1)
# dt2 = convertTime(str2)
# dtr = (dt2 - dt1) 
# dtr.seconds, dt1

In [15]:
%%time
df['connStart'] = (df['gprsStatus_lastConnStart']).apply(lambda x: convertTime(x))
df['connStop'] = (df['gprsStatus_lastConnStop']).apply(lambda x: convertTime(x))
df['session_time_seconds'] = (df['connStop'] - df['connStart']).apply(lambda x: x / np.timedelta64(1, 's'))

CPU times: user 1min 2s, sys: 310 ms, total: 1min 2s
Wall time: 1min 2s


In [16]:
df.drop(['gprsStatus_lastConnStart', 'gprsStatus_lastConnStop'], axis=1, inplace=True)

In [17]:
df.columns

Index(['icc', 'imsi', 'msisdn', 'lifeCycleStatus', 'currentApn', 'currentIp',
       'apn_apn1', 'apn_apn2', 'apn_apn3', 'apn_apn4', 'apn_apn5', 'apn_apn6',
       'apn_apn7', 'apn_apn8', 'apn_apn9', 'apn_apn10', 'connStart',
       'connStop', 'session_time_seconds'],
      dtype='object')

In [18]:
df = df[['icc', 'imsi', 'msisdn', 'lifeCycleStatus', 'currentApn', 'currentIp',
         'connStart', 'connStop', 'session_time_seconds',
         'apn_apn1','apn_apn2', 'apn_apn3', 'apn_apn4', 'apn_apn5', 
         'apn_apn6','apn_apn7', 'apn_apn8', 'apn_apn9', 'apn_apn10']]

In [19]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
session_time_seconds,"1,899,222.00","8,321.35","372,037.40","-72,049,352.29",16.98,665.00,"5,357.00","55,293,234.68"


In [20]:
# df.head()

### SIMs connected and blocked!

In [21]:
df_notActivated_connected = df[
    (df['lifeCycleStatus'] != 'ACTIVATED') &
    (df['currentApn'] != 'nan') 
]
df_notActivated_connected.shape

(2274, 19)

In [22]:
pd.DataFrame(df_notActivated_connected['lifeCycleStatus'].value_counts())

,lifeCycleStatus
SUSPENDED,2274


In [23]:
# df_notActivated_connected.head()

### SIMs without any APN

In [40]:
df_no_apn = df[
    (df['apn_apn1'] == 'nan') &
    (df['apn_apn2'] == 'nan') &
    (df['apn_apn3'] == 'nan') &
    (df['apn_apn4'] == 'nan') &
    (df['apn_apn5'] == 'nan') &
    (df['apn_apn6'] == 'nan') &
    (df['apn_apn7'] == 'nan') &
    (df['apn_apn8'] == 'nan') &
    (df['apn_apn9'] == 'nan') &
    (df['apn_apn10'] == 'nan')
  ].copy()

df_no_apn.shape

(39404, 19)

In [48]:
# df_no_apn.head()

In [47]:
pd.DataFrame(df_no_apn['currentApn'].value_counts())

,currentApn
nan,39404


In [37]:
df_no_apn.head()

,icc,imsi,msisdn,lifeCycleStatus,currentApn,currentIp,connStart,connStop,session_time_seconds,apn_apn1,apn_apn2,apn_apn3,apn_apn4,apn_apn5,apn_apn6,apn_apn7,apn_apn8,apn_apn9,apn_apn10
20,89551080337004409400,724108002436232,5511973875339,SUSPENDED,pagseguro.m2m.vivo.com.br,10.169.181.203,2020-06-03 04:16:38,2020-06-01 22:51:45,"-105,893.00",apn.pagseguro.uol,pagseguro.vivo.com.br,nan,nan,nan,nan,nan,nan,nan,nan
51,89551080337004479916,724108002443283,5511975396145,ACTIVATED,pagseguro.m2m.vivo.com.br,10.170.177.67,2021-06-07 01:27:00,2021-06-07 01:26:37,-23.00,apn.pagseguro.uol,pagseguro.vivo.com.br,nan,nan,nan,nan,nan,nan,nan,nan
60,89551080337004406893,724108002435981,5511942393174,ACTIVATED,pagseguro.m2m.vivo.com.br,10.168.193.162,2021-06-04 15:30:51,2021-06-04 15:29:53,-58.00,apn.pagseguro.uol,pagseguro.vivo.com.br,nan,nan,nan,nan,nan,nan,nan,nan
68,89551080337004478033,724108002443095,5511956197674,ACTIVATED,pagseguro.m2m.vivo.com.br,10.171.37.66,2021-06-08 11:08:39,2021-06-07 21:00:53,"-50,866.00",apn.pagseguro.uol,pagseguro.vivo.com.br,nan,nan,nan,nan,nan,nan,nan,nan
85,89551080337004541616,724108002449453,5511941595007,ACTIVATED,pagseguro.m2m.vivo.com.br,10.171.70.106,2021-06-08 05:56:52,2021-06-08 05:56:48,-4.00,apn.pagseguro.uol,pagseguro.vivo.com.br,nan,nan,nan,nan,nan,nan,nan,nan


### SIMs with at least one APN

https://stackoverflow.com/questions/44706485/how-to-remove-rows-in-a-pandas-dataframe-if-the-same-row-exists-in-another-dataf

In [25]:
%%time
df_apn = pd.merge(df, df_no_apn, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
df_apn.shape

CPU times: user 13.9 s, sys: 972 ms, total: 14.9 s
Wall time: 13.4 s


(1859818, 19)

In [26]:
# validation
df_apn.shape[0] == df.shape[0] - df_no_apn.shape[0]

True

### Public APN

SIMs connected in APN smart.m2m.vivo.com.br without it is be configured in Kite

In [53]:
df_public = df[
    (df['currentApn'] == 'smart.m2m.vivo.com.br') &
    (df['apn_apn1'] != 'smart.m2m.vivo.com.br') &
    (df['apn_apn2'] != 'smart.m2m.vivo.com.br') &
    (df['apn_apn3'] != 'smart.m2m.vivo.com.br') &
    (df['apn_apn4'] != 'smart.m2m.vivo.com.br') &
    (df['apn_apn5'] != 'smart.m2m.vivo.com.br') &
    (df['apn_apn6'] != 'smart.m2m.vivo.com.br') &
    (df['apn_apn7'] != 'smart.m2m.vivo.com.br') &
    (df['apn_apn8'] != 'smart.m2m.vivo.com.br') &
    (df['apn_apn9'] != 'smart.m2m.vivo.com.br') &
    (df['apn_apn10'] != 'smart.m2m.vivo.com.br')
]
df_public.shape

(1204, 19)

In [54]:
df_public.head()

,icc,imsi,msisdn,lifeCycleStatus,currentApn,currentIp,connStart,connStop,session_time_seconds,apn_apn1,apn_apn2,apn_apn3,apn_apn4,apn_apn5,apn_apn6,apn_apn7,apn_apn8,apn_apn9,apn_apn10
5554,89551080137015075830,724108002974037,5511973056396,SUSPENDED,smart.m2m.vivo.com.br,10.193.253.254,2020-04-16 23:46:52,2020-04-16 23:43:37,-195.00,pagseguro.vivo.com.br,apn.pagseguro.uol,apnsmart.pagseguro.uol,nan,nan,nan,nan,nan,nan,nan
8595,89551080137015451296,724108003011583,5511955897197,ACTIVATED,smart.m2m.vivo.com.br,10.192.194.170,2020-03-14 13:48:56,2020-03-02 14:06:48,"-1,035,728.00",pagseguro.vivo.com.br,apn.pagseguro.uol,apnsmart.pagseguro.uol,nan,nan,nan,nan,nan,nan,nan
9144,89551080137015470262,724108003013480,5511955852331,SUSPENDED,smart.m2m.vivo.com.br,10.100.228.75,2020-05-02 16:40:19,2020-05-02 16:40:14,-5.00,pagseguro.vivo.com.br,apn.pagseguro.uol,apnsmart.pagseguro.uol,nan,nan,nan,nan,nan,nan,nan
9223,89551080137015416372,724108003008091,5511955965159,SUSPENDED,smart.m2m.vivo.com.br,10.194.68.33,2020-03-15 21:34:22,2020-03-15 21:32:41,-101.00,pagseguro.vivo.com.br,apn.pagseguro.uol,nan,nan,nan,nan,nan,nan,nan,nan
9476,89551080137015406522,724108003007106,5511955927720,SUSPENDED,smart.m2m.vivo.com.br,172.18.162.29,2020-03-15 20:07:36,2020-03-15 08:36:28,"-41,468.00",pagseguro.vivo.com.br,apn.pagseguro.uol,apnsmart.pagseguro.uol,nan,nan,nan,nan,nan,nan,nan


## Output

### Output file

In [27]:
def getCurrentDate():
    local_dt = datetime.now()
    return str(local_dt.year) +'.'+ str(local_dt.month) + '.' + str(local_dt.day) + '_' + str(local_dt.hour) + 'h' + str(local_dt.minute)

In [28]:
!mkdir data/result

mkdir: cannot create directory ‘data/result’: File exists


In [29]:
df_no_apn.to_csv('data/result/pagseguro_no_apn_' + getCurrentDate() + '.csv')

In [30]:
df_apn.to_csv('data/result/pagseguro_with_apn_' + getCurrentDate() + '.csv')

In [31]:
df_notActivated_connected.to_csv('data/result/pagseguro_notActivated_connected_' + getCurrentDate() + '.csv')

In [55]:
df_public.to_csv('data/result/pagseguro_publicAPN_' + getCurrentDate() + '.csv')

In [32]:
df.to_csv('data/result/pagseguro_all_' + getCurrentDate() + '.csv')

In [56]:
!ls data/result/*.csv -lht

-rw-rw-r-- 1 rodrigo rodrigo 287K jun 10 15:03 data/result/pagseguro_publicAPN_2021.6.10_15h3.csv
-rw-rw-r-- 1 rodrigo rodrigo 402M jun 10 14:46 data/result/pagseguro_all_2021.6.10_14h46.csv
-rw-rw-r-- 1 rodrigo rodrigo 564K jun 10 14:46 data/result/pagseguro_notActivated_connected_2021.6.10_14h46.csv
-rw-rw-r-- 1 rodrigo rodrigo 396M jun 10 14:46 data/result/pagseguro_with_apn_2021.6.10_14h45.csv
-rw-rw-r-- 1 rodrigo rodrigo 6,9M jun 10 14:45 data/result/pagseguro_no_apn_2021.6.10_14h45.csv
